In [1]:
import logging
import os
import sys
from io import BytesIO
import pandas as pd
import requests
from datetime import date, timedelta, datetime
from collections import Counter
import numpy as np
import json
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import time
from astropy.table import Table

In [2]:
from desimodel.footprint import is_point_in_desi

In [3]:
TNS_API_KEY= '006e7e40b06030997b93b0d0cf0973e56c408875'  # Works on real and sandbox now
TNS_BOT_NAME = 'DESIRT_Bot'
TNS_BOT_ID = 105220

In [4]:
CATALOG_URL = 'https://www.wis-tns.org/system/files/tns_public_objects/tns_public_objects.csv.zip'

In [5]:
def download_table():
    with requests.post(
        CATALOG_URL,
        headers={
            'user-agent': 'tns_marker{{"tns_id":"{id}","type": "bot", "name":"{name}"}}'.format(
                id=TNS_BOT_ID,
                name=TNS_BOT_NAME,
            )
        },
        data={
            'api_key': (None, TNS_API_KEY)
        },
    ) as response:
        return response.content

In [6]:
data = download_table()

In [7]:
df = pd.read_csv(
        BytesIO(data),
        skiprows=1,
        compression='zip',
    )

In [8]:
date.today()

datetime.date(2024, 12, 16)

In [9]:
df["creationdate"] = pd.to_datetime(df["creationdate"])
df["lastmodified"] = pd.to_datetime(df["lastmodified"])

In [10]:
now = datetime.now()
last_week = now - timedelta(days=7)

In [11]:
last_week_rows = df[df['lastmodified'] > last_week]

In [12]:
last_week_rows.columns

Index(['objid', 'name_prefix', 'name', 'ra', 'declination', 'redshift',
       'typeid', 'type', 'reporting_groupid', 'reporting_group',
       'source_groupid', 'source_group', 'discoverydate', 'discoverymag',
       'discmagfilter', 'filter', 'reporters', 'time_received',
       'internal_names', 'Discovery_ADS_bibcode', 'Class_ADS_bibcodes',
       'creationdate', 'lastmodified'],
      dtype='object')

In [13]:
last_week_visible = last_week_rows.iloc[np.where(last_week_rows["declination"] > -30)]

In [14]:
t = Table.read("/global/cfs/cdirs/desi/survey/ops/surveyops/trunk/ops/tiles-main.ecsv")
sel = (t["IN_DESI"]) & (t["STATUS"] == "unobs")
sel &= (t["PROGRAM"] == "BRIGHT") | (t["PROGRAM"] == "DARK")
t = t[sel]

In [15]:
last_week_visible_tile_tf = is_point_in_desi(t, last_week_visible["ra"], last_week_visible["declination"])

In [16]:
last_week_visible_tile = last_week_visible.iloc[last_week_visible_tile_tf]

In [17]:
def is_close(distance, brightness):
    print(distance)
    print(brightness)
    print((10 / (brightness - 3)))
    print(np.log10(distance))
    return ((10 / (brightness - 3)) > np.log10(distance))

In [18]:
def get_bright_star(sky_object, cone = 5*u.arcsec):
    result = Vizier.query_region(sky_object,
                             radius=cone,
                             catalog='I/355/gaiadr3',
                             column_filters={'Gmag': '<15'})
    if len(result) == 0:
        return False
    return len(result[0]) > 0

In [19]:
too_close = []
for i in tqdm(SkyCoord(last_week_visible_tile["ra"], last_week_visible_tile["declination"], frame = "icrs", equinox = 'J2000', unit="deg")):
    too_close.append(get_bright_star(i))
    time.sleep(2.5)

100%|██████████| 130/130 [06:34<00:00,  3.03s/it]


In [20]:
too_close = np.asarray(too_close)

In [21]:
last_week_visible_seperate = last_week_visible_tile.iloc[np.where(too_close == False)]

In [22]:
Counter(last_week_visible_seperate["reporting_group"])

Counter({'ZTF': 48,
         'Pan-STARRS': 23,
         'ATLAS': 17,
         'ALeRCE': 12,
         'GaiaAlerts': 11,
         'GOTO': 10,
         'XOSS': 4,
         'SNHunt': 1,
         'GEOTS': 1,
         nan: 1,
         'BlackGEM': 1,
         'MASTER': 1})

In [23]:
last_week_visible_seperate.iloc[np.where(last_week_visible_seperate["reporting_group"] == "ZTF")]["internal_names"]

4                                           ZTF24abxylal
12                                          ZTF24abxypih
15                                          ZTF24abxyfev
41                                 ZTF24abutata, PS24mmu
43                                 ZTF24abwlhuv, PS24mpa
56                                 ZTF24abvgcga, PS24mmy
82                                 ZTF24abtmuoi, PS24mou
93          ZTF24absvdpn, ATLAS24qgz, GOTO24ikz, PS24mqf
94                     ZTF24absmvsz, ATLAS24qud, PS24moq
97          ZTF24absmnsk, ATLAS24quj, GOTO24iqa, PS24mml
100                    ZTF24abrmdti, ATLAS24qge, PS24mpc
101                                ZTF24abricyl, PS24mpb
102         ZTF24abribze, GOTO24idm, ATLAS24qhu, PS24mpq
110         ZTF24aboiurx, ATLAS24pql, GOTO24hvc, PS24mof
114         ZTF24ablzlmd, ATLAS24phv, GOTO24hsl, PS24mnu
144                                         ZTF24abxxfzd
151                                         ZTF24abxvvfi
167         ZTF24abwlgza, ATLAS

In [24]:
last_week_visible_seperate

,objid,name_prefix,name,ra,declination,redshift,typeid,type,reporting_groupid,reporting_group,...,discoverymag,discmagfilter,filter,reporters,time_received,internal_names,Discovery_ADS_bibcode,Class_ADS_bibcodes,creationdate,lastmodified
3,168397,AT,2024aebu,239.887917,38.137581,NaN,NaN,NaN,24.0,GaiaAlerts,...,18.5100,75.0,G,"S.T. Hodgkin, E. Breedt, A. Delgado, D.L. Harr...",2024-12-15 17:35:11,Gaia24dqr,NaN,NaN,2024-12-15 17:35:12,2024-12-15 17:35:12
4,168396,AT,2024aebt,135.097017,64.861455,NaN,NaN,NaN,48.0,ZTF,...,19.5589,110.0,g,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-12-15 17:05:51,ZTF24abxylal,NaN,NaN,2024-12-15 17:05:52,2024-12-15 17:05:52
6,168394,AT,2024aebr,20.880884,8.795317,NaN,NaN,NaN,4.0,Pan-STARRS,...,20.0000,26.0,w,"K. C. Chambers, T. de Boer, J. Fairlamb, M. Hu...",2024-12-15 16:55:31,PS24mqe,NaN,NaN,2024-12-15 16:55:32,2024-12-15 16:55:32
7,168393,AT,2024aebq,129.708309,18.904488,NaN,NaN,NaN,4.0,Pan-STARRS,...,19.3400,26.0,w,"K. C. Chambers, T. de Boer, J. Fairlamb, M. Hu...",2024-12-15 16:25:11,PS24mpt,NaN,NaN,2024-12-15 16:25:12,2024-12-15 16:25:12
8,168392,AT,2024aebp,203.561293,-6.754273,NaN,NaN,NaN,74.0,ALeRCE,...,19.0586,111.0,r,"F. Forster, F.E. Bauer, G. Pignata, A. Munoz-A...",2024-12-15 15:44:33,ZTF24abxypwh,NaN,NaN,2024-12-15 15:44:34,2024-12-15 15:44:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,168185,AT,2024adtq,229.261914,12.635093,NaN,NaN,NaN,74.0,ALeRCE,...,19.7914,111.0,r,"A. Munoz-Arancibia, G. Pignata, F.E. Bauer, F....",2024-12-09 17:26:11,ZTF24abxgkcy,2024TNSTR4821....1M,NaN,2024-12-09 17:26:19,2024-12-09 17:26:19
424,168100,AT,2024adqj,184.738299,36.937146,NaN,NaN,NaN,48.0,ZTF,...,19.9771,110.0,g,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-12-08 12:14:13,ZTF24abxfuog,2024TNSTR4813....1S,NaN,2024-12-08 12:14:14,2024-12-09 17:26:21
437,166671,SN,2024aboy,207.144875,24.893439,0.029716,13.0,SN IIn,18.0,ATLAS,...,18.2970,72.0,orange,"J. Tonry, L. Denneau, H. Weiland, R. Siverd (I...",2024-11-20 21:15:08,"ATLAS24qhq, ZTF24abuvisc, Gaia24dmf, GOTO24ivg",2024TNSTR4550....1T,2024TNSCR4840....1S,2024-11-20 21:15:10,2024-12-09 19:17:26
438,166310,SN,2024abbi,359.060582,12.483990,0.100000,3.0,SN Ia,48.0,ZTF,...,19.9649,110.0,g,"Jesper Sollerman (SU), Christoffer Fremling (C...",2024-11-14 05:15:34,"ZTF24abrkfol, ATLAS24qhv, PS24llr",2024TNSTR4497....1S,2024TNSCR4840....1S,2024-11-14 05:15:35,2024-12-09 19:31:34


In [25]:
now.date().year

2024

In [26]:
last_week_visible_seperate.to_csv("TNS_ToO/ToO_Objects-{}{:02d}{:02d}.csv".format(now.date().year, now.date().month, now.date().day), index = False)

source /global/cfs/cdirs/desi/software/desi_environment.sh 23.1

In [27]:
print("python build_ledger.py TNS_ToO/ToO_Objects-{}{:02d}{:02d}.csv -o TNS_ToO/ToO-input-{}{:02d}{:02d}.ecsv".format(now.date().year, now.date().month, now.date().day, now.date().year, now.date().month, now.date().day))

python build_ledger.py TNS_ToO/ToO_Objects-20241216.csv -o TNS_ToO/ToO-input-20241216.ecsv


cd /pscratch/sd/a/adamyers/ToO/segev_put_it_here

In [28]:
print("cat ~/GitHub/DESI-ALERCE/TNS_ToO/ToO-input-{}{:02d}{:02d}.ecsv >> ToO-input.ecsv".format(now.date().year, now.date().month, now.date().day))

cat ~/GitHub/DESI-ALERCE/TNS_ToO/ToO-input-20241216.ecsv >> ToO-input.ecsv
